# Agenda

1. What is Polars? What is its relationship with Pandas and the rest of the data ecosystem in Python?
2. Series
3. Data frames
4. Reading in CSV files
5. dtypes
6. Expressions
7. Selecting rows with `df.select`
8. Selecting columns
9. `df.with_columns`
10. `df.filter`
11. Sorting
12. Grouping
13. Optimizing queries and "lazy frames"

# What is Polars?

Data frames are everywhere, but they aren't native to very many languages. We're seeing a growing number of libraries that provide data frames outside of the core of languages. The bad news is that you have to install / choose which data-frame package you want to use in Python. The good news is that we have a few great options to choose from.

Pandas is the 900-pound gorilla in this space.  Polars is a relatively new entry, and it's written for (a) speed of execution, (b) builtin multithreading, (c) lazy loading of data, and (d) a very minimalist, elegant API.

The URL is pola.rs, because it's written in the Rust language.  The Python API that Polars exposes is very similar to the Pandas API, making it fairly easy for someone to move from Pandas to Polars.

I personally see Polars as great for when you have tons of data and/or execution speed is critical. If you're a snob who wants a very elegant API, then it's great, too. 



# Installing Polars

You can install it from PyPI using `pip`:

    pip install -U polars

You'll probably want to install the complete Polars package and dependencies and extras, using

    pip install -U 'polars[all]'

If you're using a Mac with Apple Silicon, Polars might die on you because your copy of Python was compiled for Intel, and is in compatibility mode.  If you're like me, and can't/won't/don't know how to recompile Python for Apple Silicon, you can just install a different Polars package from PyPI that takes this into account:

    pip install -U 'polars-lts-cpu[all]'
   

In [1]:
import polars as pl    # just as we use Pandas as pd, we use Polars as pl

# Series

A series is a 1D data structure, just like in Pandas. (There, behind the scenes, we have a NumPy array. That is *not* true in Polars!) 

We can create a Polars series just by invoking `pl.Series` on a list of values. Polars will (like Pandas) figure out what kinds of values we have, and set the dtype.

In [8]:
s = pl.Series(
    values=[10, 20, 30, 40, 50],
    name='numbers'
)

In [9]:
s

numbers
i64
10
20
30
40
50


# Some differences

1. No index! There is no index in Polars, period.
2. No name, although we can give it a name by passing the `name` argument, which comes *before* the data, unless you want both `name` and `values` to be keyword arguments, in which case they can be in any order.
3. The shape is always displayed at the top of the series (or data frame)
4. The dtype is displayed at the top of the series. And here, we see that the dtype is `i64`, aka `int64`

In [10]:
s = pl.Series(
    values=[10.5, 20.5, 30.5, 40.5, 50.5],
    name='numbers'
)

s

numbers
f64
10.5
20.5
30.5
40.5
50.5


In [11]:
# let's try another dtype

s = pl.Series(
    values=[10.5, 20.5, 30.5, 40.5, 50.5],
    name='numbers',
    dtype=pl.Float32   # use Polars dtypes, not NumPy dtypes and not strings
)

s

numbers
f32
10.5
20.5
30.5
40.5
50.5


In [18]:
# What if I have a mixture of types?

s = pl.Series(
    values=[10, 20.5, 30, 40.5, 50],
    dtype=pl.Int64
)

s

TypeError: unexpected value while building Series of type Int64; found value of type Float64: 20.5

Hint: Try setting `strict=False` to allow passing data with mixed types.

In [19]:
s = pl.Series(
    values=[10, 20.5, 30, 40.5, 50],
    dtype=pl.Int64,
    strict=False
)

s

""
i64
10
20
30
40
50


In [21]:
s = pl.Series(
    'hello out there from polars world!'.split()
)

s

""
str
"""hello"""
"""out"""
"""there"""
"""from"""
"""polars"""
"""world!"""


In [22]:
s.dtype

String

In [26]:
s1 = pl.Series(
    values=[10, 20, 30, 40, 50]
)

s
    

""
i64
10
20
30
40
50


In [27]:
s2 = pl.Series(
    values=[100, 200, 300, 400, 500]
)

s2
    

""
i64
100
200
300
400
500


In [28]:
# if we perform an operation on two series, assuming that they are both the same 
# length (i.e., shape), then the operations will be per

s1 + s2

""
i64
110
220
330
440
550
